In [33]:
import requests
from datetime import datetime, timezone
from pprint import pprint

from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()

# Now you can access your API keys directly
NOTION_TOKEN = os.environ.get("NOTION_TOKEN")
DATABASE_ID = os.environ.get("NOTION_DATABASE_ID")

headers = {
    "Authorization": "Bearer " + NOTION_TOKEN,
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28",
}

def create_page(data: dict):
    create_url = "https://api.notion.com/v1/pages"

    payload = {"parent": {"database_id": DATABASE_ID}, "properties": data}

    res = requests.post(create_url, headers=headers, json=payload)
    # print(res.status_code)
    return res

def get_pages(num_pages=None):
    """
    If num_pages is None, get all pages, otherwise just the defined number.
    """
    url = f"https://api.notion.com/v1/databases/{DATABASE_ID}/query"

    get_all = num_pages is None
    page_size = 100 if get_all else num_pages

    payload = {"page_size": page_size}
    response = requests.post(url, json=payload, headers=headers)

    data = response.json()

    # Comment this out to dump all data to a file
    import json
    with open('db.json', 'w', encoding='utf8') as f:
       json.dump(data, f, ensure_ascii=False, indent=4)

    results = data["results"]
    while data["has_more"] and get_all:
        payload = {"page_size": page_size, "start_cursor": data["next_cursor"]}
        url = f"https://api.notion.com/v1/databases/{DATABASE_ID}/query"
        response = requests.post(url, json=payload, headers=headers)
        data = response.json()
        results.extend(data["results"])

    return results

print(get_pages())

[{'object': 'page', 'id': '24c94cc6-e4ba-815f-a794-c98606f7d177', 'created_time': '2025-08-11T05:51:00.000Z', 'last_edited_time': '2025-08-11T05:51:00.000Z', 'created_by': {'object': 'user', 'id': '214da348-f632-4e03-8e71-a371b7b55dc5'}, 'last_edited_by': {'object': 'user', 'id': '214da348-f632-4e03-8e71-a371b7b55dc5'}, 'cover': None, 'icon': None, 'parent': {'type': 'database_id', 'database_id': '24b94cc6-e4ba-8082-92ce-c1821f5473ac'}, 'archived': False, 'in_trash': False, 'properties': {'Call Number': {'id': 'HK_m', 'type': 'rich_text', 'rich_text': [{'type': 'text', 'text': {'content': '306.73', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': '306.73', 'href': None}]}, 'Library Location': {'id': 'HWa%60', 'type': 'rich_text', 'rich_text': [{'type': 'text', 'text': {'content': 'Sunnyvale Public Library', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethr

# Query the database

In [38]:
pages = get_pages()

for page in pages:
    props = page["properties"]

    data = {
        "ID": page["id"],
        "Title": props["Title"]["title"][0]["text"]["content"],
        "Status": props["Status"]["status"]["name"],
        "Call Number": props["Call Number"]["rich_text"][0]["text"],
        "Location": props["Library Location"]["rich_text"][0]["text"]["content"],
        "Date": props["Date"]["date"]["start"]
    }

    pprint(data)  # pretty print dict

{'Call Number': {'content': '306.73 URY', 'link': None},
 'Date': '2025-08-11T05:54:00.000+00:00',
 'ID': '24c94cc6-e4ba-816b-a8c2-de9c7fe2b4ad',
 'Location': 'Sunnyvale Public Library',
 'Status': 'All Copies in Use',
 'Title': 'How to Not Die Alone'}
{'Call Number': {'content': '615.222', 'link': None},
 'Date': '2025-08-09',
 'ID': '24b94cc6-e4ba-80b4-9e94-c842734febe3',
 'Location': 'Sunnyvale Public Library',
 'Status': 'All Copies in Use',
 'Title': 'Maybe you should talk to somone'}


# Create database

In [27]:
def create_page(data: dict):
    create_url = "https://api.notion.com/v1/pages"

    payload = {"parent": {"database_id": DATABASE_ID}, "properties": data}

    # Debug: Print payload
    print("Payload:", payload)
    print("Headers:", headers)

    res = requests.post(create_url, headers=headers, json=payload)
    # print(res.status_code)
    # Debug: Print response details
    print(f"Status: {res.status_code}")
    print(f"Response: {res.text}")
    
    if res.status_code != 200:
        print(f"Error: {res.status_code} - {res.text}")
    return res

In [36]:
from datetime import datetime, timezone

title = "How to Not Die Alone"
date_str = datetime.now().astimezone(timezone.utc).isoformat()
status = "All Copies in Use"
call_number = "306.73 URY"  # Must be a number, not string unless I change the datatype in Notion
location = "Sunnyvale Public Library"
data = {
    "Title": {
        "title": [{"text": {"content": title}}]
    },
    "Status": {
        "status": {"name": status}  # must match an existing Status option in Notion
    },
    "Call Number": {
        "rich_text": [{"text": {"content": call_number}}]
    },
    "Library Location": {
        "rich_text": [{"text": {"content": location}}]
    },
    "Date": {
        "date": {"start": date_str, "end": None}
    }
}
create_page(data)

<Response [200]>